In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Show code", True: "Hide code"}


def toggle_code(state):

    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)

    display(HTML(output))


def button_action(value):

    """
    Calls the toggle_code function and updates the button description.
    """

    state = value.new

    toggle_code(state)

    value.owner.description = button_descriptions[state]


state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")

display(button)

In [ ]:
import os
import re
import string
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import numpy as np
from sklearn import datasets
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from scipy.special import expit
import warnings 

In [ ]:
warnings.filterwarnings ("ignore")

# Parte I

## 1

Luego de explorar el diseño de registro de la base de hogar, creemos que las siguiens variables puedenn ser muy predictivas de pobreza:

IV11 = El desague del baño es:

                  1. a red pública (cloaca)
                  2. a cámara séptica y pozo ciego 
                  3. solo a pozo ciego 
                  4. a hoyo/excavación en la tierra
                  
IV12_3 = La vivienda está ubicada en villa de emergencia (por observación)

                  1 = Sí
                  2 = No

II7 = Régimen de tenencia: 

               01 = Propietario de la vivienda y el terreno
               02 = Propietario de la vivienda solamente
               03 = Inquilino / arrendatario de la vivienda
               04 = Ocupante por pago de impuestos / expensas
               05 = Ocupante en relación de dependencia
               06 = Ocupante gratuito (con permiso)
               07 = Ocupante de hecho (sin permiso)
               08 = Está en sucesión

II8 = Combustible utilizado para cocinar:

                 01 = Gas de red
                 02 = Gas de tubo / garrafa
                 03 = Kerosene / leña / carbón

II9 = Baño (tenencia y uso): 
   
        01 = Uso exclusivo del hogar
        02 = Compartido con otro/s hogar/es de la misma vivienda
        03 = Compartido con otra/s vivienda/s
        04 = No tiene baño
        
V5 = ¿En los últimos tres meses, las personas de este hogar han vivido de subsidio o ayuda social (en dinero)del gobierno, iglesias, etc.?: 

                  1 = Sí
                  2 = No

V17 = ¿En los últimos tres meses, han tenido que vender alguna de sus pertenencias?:

                  1 = Sí
                  2 = No

## 2

In [ ]:
# Cargo el excel de la EPH
eph_hogar = pd.read_excel("usu_hogar_T121.xlsx")
eph_indiv = pd.read_excel("usu_individual_T121.xlsx")

In [ ]:
# Me quedo solo con los aglomerados de CABA y GBA
eph_hogar.drop(eph_hogar[(eph_hogar['AGLOMERADO'] != 32) & (eph_hogar["AGLOMERADO"] != 33)].index, inplace = True)
eph_hogar.reset_index(inplace = True, drop = True)
eph_indiv.drop(eph_indiv[(eph_indiv['AGLOMERADO'] != 32) & (eph_indiv["AGLOMERADO"] != 33)].index, inplace = True)
eph_indiv.reset_index(inplace = True, drop = True)

## 3

In [ ]:
# Junto la eph de hogares y individuos por la columna CODUSU, el suffixes = ("", "_y") es para que distinga las columnas
#repetidas en ambos dataframes colocandolo el sufijo "y" a los nombres de las columnas de la eph_indiv
eph = eph_hogar.merge(eph_indiv, on = "CODUSU", suffixes = ("", "_y") , how = "outer")
eph = eph[eph.columns.drop(list(eph.filter(regex='_y')))]

<span style="color:red"> (Rodrigo) Aquí se debería unir por "CODUSU" y "NRO_HOGAR", ya que ambos definen un hogar; "CODUSU" sólo define una vivienda, que puede contener muchos hogares. Sin embargo, la diferencia es mínima ya que por hacerlo así sólo pierden 9 observaciones, así que la influencia en los resultados que obtienen debería ser despreciable.</span>

## 4

In [ ]:
def limpiar_eph(eph):

    '''
        función para limpiar la eph
        input:
            eph: Variable que tenga almacenada la eph sin limpiar

        output:
            eph: Variable que tiene almacenada la eph limpia
    '''
    
    eph.dropna(thresh = 1, inplace = True) # Si tiene todos NA, la fila se va
    
    eph.dropna(thresh = 1, axis = "columns", inplace = True) # Si la columna tiene todos NA, la columna se va
    
    try:
        # Si no lo defino como uno me dropea un montón de filas que tienen NaN
        eph.PP08D1 = eph.PP08D1.fillna(0)
        eph.PP08F2 = eph.PP08F2.fillna(0)
        # dropeo observaciones con valores que no tienen sentido. CH06 = edad, PP08D1, P21, P47T, ITF y IPCF son 
        # todas variables de ingresos, no pueden ser menores a 0
        eph.drop(eph[(eph["CH06"] < 0) | (eph["PP08D1"] < 0) | (eph["P21"] < 0) | 
                     (eph["P47T"] < 0) | (eph["ITF"] < 0) | (eph["IPCF"] < 0) | 
                     (eph["PP08F2"] < 0)].index, inplace = True)
        eph["CH14"].fillna(99, inplace = True) #Muchos no respondieron entonces reemplazamos los missing por 99.
        eph.fillna(0, inplace = True) # Relleno todos con 0 los missing de las columnas que representan subcategorías en
        #las que solo respondieron algunos. 
        #Para las siguientes variables dummys hay entre 15 y 20 observaciones para las que toma valor 9, es decir Ns/Nr.
        #Como son pocas observaciones y siempre las mismas, optamos por eliminarlas. 
        eph.drop(eph[(eph["V1"] == 9 ) | (eph["V2"] == 9) | (eph["V3"] == 9) | (eph["V4"] == 9) | (eph["V5"] == 9) | 
                     (eph["V6"] == 9 ) | (eph["V7"] == 9) | (eph["V8"] == 9) | (eph["V9"] == 9) | (eph["V10"] == 9) | 
                     (eph["V11"] == 9) | (eph["V12"] == 9) | (eph["V13"] == 9) | (eph["V14"] == 9) | (eph["V15"] == 9) |
                     (eph["V16"] == 9) | (eph["V17"] == 9) | (eph["V18"] == 9) | (eph["V19_A"] == 9) | 
                     (eph["V19_B"] == 9)].index, inplace = True)
        eph.reset_index(inplace = True, drop = True)  
    except:
        print("No se encontró alguna columna, revisar")      

## 5

In [ ]:
limpiar_eph(eph)

## 6 (hay que correr el codigo hasta el punto 9 iclusive antes de correr el codigo de este punto)

Las variables a analizar son 5 de las mencionadas en el punto 1: 
    
    IV12_3, II8, V5, V17 y II9

In [ ]:
#IV12_3 = La vivienda está ubicada en villa de emergencia (por observación)
eph["IV12_3"].describe()

In [ ]:
# pobres_villa = proporción de pobres que viven en villas de emergencia. 
# pobres_no_villa = proporción de pobres que no viven en villas de emergencia.
pobres_villa = respondieron[(respondieron["pobre"] == 1) & (respondieron["IV12_3"] == 1)].IV12_3.apply("count")
pobres_no_villa = respondieron[(respondieron["pobre"] == 1) & (respondieron["IV12_3"] == 2)].IV12_3.apply("count")
pobres_villa = pobres_villa/(respondieron[respondieron["pobre"] == 1].pobre.apply("count"))
pobres_no_villa = pobres_no_villa/(respondieron[respondieron["pobre"] == 1].pobre.apply("count"))
# villa = proporción de no pobres que viven en villas de emergencia. 
# no_villa = proporción de no pobres que no viven en villas de emergencia.
villa = respondieron[(respondieron["pobre"] == 0) & (respondieron["IV12_3"] == 1)].IV12_3.apply("count")
no_villa = respondieron[(respondieron["pobre"] == 0) & (respondieron["IV12_3"] == 2)].IV12_3.apply("count")
villa = villa/(respondieron[respondieron["pobre"] == 0].pobre.apply("count"))
no_villa = no_villa/(respondieron[respondieron["pobre"] == 0].pobre.apply("count"))
# Gráfico de barras que compara la proporción de pobres y no pobres que viven en villas de emergencia.
plt.bar("pobres", pobres_villa)
plt.bar("no pobres", villa)
plt.title("% de personas en villas de emergencia")
plt.show

In [ ]:
eph["II8"].describe()

In [ ]:
# pobres_gas_tubo = proporción de pobres que utilizan gas de tubo. 
# pobres_gas_red = proporción de pobres que utilizan gas de red.
pobres_gas_tubo = respondieron[(respondieron["pobre"] == 1) & (respondieron["II8"] == 2)].II8.apply("count")
pobres_gas_red = respondieron[(respondieron["pobre"] == 1) & (respondieron["II8"] == 1)].II8.apply("count")
pobres_gas_tubo = pobres_gas_tubo/(respondieron[respondieron["pobre"] == 1].pobre.apply("count"))
pobres_gas_red = pobres_gas_red/(respondieron[respondieron["pobre"] == 1].pobre.apply("count"))
# gas_tubo = proporción de no pobres que que utilizan gas de tubo.
# gas_red = proporción de no pobres que que utilizan gas de red.
gas_tubo = respondieron[(respondieron["pobre"] == 0) & (respondieron["II8"] == 2)].II8.apply("count")
gas_red = respondieron[(respondieron["pobre"] == 0) & (respondieron["II8"] == 1)].II8.apply("count")
gas_tubo = gas_tubo/(respondieron[respondieron["pobre"] == 0].pobre.apply("count"))
gas_red = gas_red/(respondieron[respondieron["pobre"] == 0].pobre.apply("count"))
# Gráfico de barras que compara la proporción de pobres y no pobres que usan garrafa.
plt.bar("pobres", pobres_gas_tubo)
plt.bar("no pobres", gas_tubo)
plt.title("% de personas que usan garrafa")
plt.show

In [ ]:
eph["V5"].describe()

In [ ]:
# pobres_plan = proporción de pobres que recibieron subsidio o ayuda social en los últimos 3 meses. 
pobres_plan = respondieron[(respondieron["pobre"] == 1) & (respondieron["V5"] == 2)].V5.apply("count")
pobres_plan = pobres_plan/(respondieron[respondieron["pobre"] == 1].pobre.apply("count"))
# plan = proporción de no pobres que recibieron subsidio o ayuda social en los últimos 3 meses.
plan = respondieron[(respondieron["pobre"] == 0) & (respondieron["V5"] == 2)].V5.apply("count")
plan = plan/(respondieron[respondieron["pobre"] == 0].pobre.apply("count"))
# Gráfico de barras que compara la proporción de pobres y no pobres que recibieron planes o ayuda social.
plt.bar("pobres", pobres_plan)
plt.bar("no pobres", plan)
plt.title("% de personas que recibieron planes recientemente")
plt.show

In [ ]:
eph["V17"].describe()

In [ ]:
# pobres_venta = proporción de pobres que tuvieron que vender pertenencias en los últimos 3 meses. 
pobres_venta = respondieron[(respondieron["pobre"] == 1) & (respondieron["V17"] == 1)].V17.apply("count")
pobres_venta = pobres_venta/(respondieron[respondieron["pobre"] == 1].pobre.apply("count"))
# venta = proporción de no pobres que tuvieron que vender pertenencias en los últimos 3 meses.
venta = respondieron[(respondieron["pobre"] == 0) & (respondieron["V17"] == 1)].V17.apply("count")
venta = venta/(respondieron[respondieron["pobre"] == 0].pobre.apply("count"))
# Gráfico de barras que compara la proporción de pobres y no pobres que tuvieron que vender pertenencias en los últimos 3 meses.
plt.bar("pobres", pobres_venta)
plt.bar("no pobres", venta)
plt.title("% de personas que vendieron pertenencias")
plt.show

In [ ]:
eph["II9"].describe()

In [ ]:
# pobres_menor = proporción de pobres que no tienen baños exclusivos.
pobres_baño_vulnerable = respondieron[(respondieron["pobre"] == 1) & (respondieron["II9"] == 2)].II9.apply("count")
pobres_baño_vulnerable += respondieron[(respondieron["pobre"] == 1) & (respondieron["II9"] == 3)].II9.apply("count")
pobres_baño_vulnerable += respondieron[(respondieron["pobre"] == 1) & (respondieron["II9"] == 4)].II9.apply("count")
pobres_baño_vulnerable = pobres_baño_vulnerable/(respondieron[respondieron["pobre"] == 1].pobre.apply("count"))
# menor = proporción de no pobres que no tienen baños exclusivos.
baño_vulnerable = respondieron[(respondieron["pobre"] == 0) & (respondieron["II9"] == 2)].II9.apply("count")
baño_vulnerable += respondieron[(respondieron["pobre"] == 0) & (respondieron["II9"] == 3)].II9.apply("count")
baño_vulnerable += respondieron[(respondieron["pobre"] == 0) & (respondieron["II9"] == 4)].II9.apply("count")
baño_vulnerable = baño_vulnerable/(respondieron[respondieron["pobre"] == 0].pobre.apply("count"))
# Gráfico de barras que compara la proporción de pobres y no pobres que no tienen baños exclusivos.
plt.bar("pobres", pobres_baño_vulnerable)
plt.bar("no pobres", 0.0001)
plt.title("% de personas sin baño exclusivo")
plt.show

In [ ]:
# Hacemos una matriz de correlaciones de las variables descriptas
correlaciones = respondieron[["IV12_3", "II8", "V5", "V17", "II9", "pobre"]].corr()
#Realizamos la matriz de correlaciones 
sns.heatmap(correlaciones, annot=True)
plt.show()

<span style="color:red"> (Rodrigo) Para variables categóricas como las elegidas no es muy informativo reportar cantidades como la media o la desviación estándar. Tiene más sentido reportar la cantidad o proporción de cada categoría (e idealmente el significado de cada código asociado), por ejemplo usando el método `value_counts` de Pandas. respondió a esa pregunta.
</span> <br><br>

## 7

In [ ]:
#Importamos la tabla de excel
tabla_adulto = pd.read_excel("tabla_adulto_equiv.xlsx", header = 3)
#Ajustamos el formato de la tabla 
tabla_adulto.dropna(axis = 1, inplace = True, thresh = 10)
tabla_adulto.dropna(inplace = True, thresh = 3)
tabla_adulto.reset_index(inplace = True, drop = True)
#Creamos la columna "adulto_equiv" como una columna de ceros
eph["adulto_equiv"] = 0
#Generamos una nueva columna llamada lista como una columna de ceros 
eph["lista"] = 0
#Iteramos por filas
#Para cada fila del data frame de acuerdo a la edad le indicamos que 
#la celda de la columna "lista" la complete con el número correspondiente 
#al índice de la tabla_adulto. 
#Por ejemplo, si la edad es menor a 30 entonces se asigna el numero 18 que es
#el índice de la fila de 18 a 29 años. 
for i in eph.index:
    if eph.loc[i, "CH06"] < 19:
        eph.loc[i, "lista"] = eph.loc[i, "CH06"]
    elif eph.loc[i, "CH06"] < 30:
        eph.loc[i, "lista"] = 18
    elif eph.loc[i, "CH06"] < 46:
        eph.loc[i, "lista"] = 19
    elif eph.loc[i, "CH06"] < 61:
        eph.loc[i, "lista"] = 20
    elif eph.loc[i, "CH06"] < 76:
        eph.loc[i, "lista"] = 21
    else:
        eph.loc[i, "lista"] = 22
#CH06 = edad, CH04 = Sexo (1 = varon y 2 = mujer)
#Para cada fila del dataframe si el individuo es varón entonces le indicamos que bajo 
#la columna "adulto_equiv" coloque la siguiente celda de tabla_adulto: 
# Fila: el índice que esta guardado en la columna "lista" en el df eph. 
# Columna: Varones 
#idem para las mujeres
#De este modo se obtienenn los valores de adulto equivalente de cada persona según 
#su sexo y edad
for i in eph.index:
    if eph.loc[i, "CH04"] == 1:
        eph.loc[i, "adulto_equiv"] = tabla_adulto.loc[eph.loc[i, "lista"], "Varones"]
    elif eph.loc[i, "CH04"] == 2:
        eph.loc[i, "adulto_equiv"] = tabla_adulto.loc[eph.loc[i, "lista"], "Mujeres"]
#Eliminamos la columna "lista"
eph.drop("lista", axis = 1, inplace = True)
#Realizamos un groupby: para cada hogar sumamos "adulto_equiv"
ad_equiv_hogar = eph[["CODUSU", "adulto_equiv"]].groupby("CODUSU").sum()
ad_equiv_hogar.reset_index(inplace = True)
#cambiamos los nombres de las columnas
ad_equiv_hogar.columns = ["CODUSU", "ad_equiv_hogar"]
#agregamos la columnna "ad_equiv_hogar" al df eph
eph = eph.merge(ad_equiv_hogar, on = "CODUSU", how = "outer")

## 8

In [ ]:
# Dividimos la muestra de la EPH en gente que no respondio ingresos totales familiares (ITF) y los que sí.
norespondieron = eph[eph["ITF"] == 0]
norespondieron.reset_index(inplace = True, drop = True)
respondieron = eph[eph["ITF"] != 0]
respondieron.reset_index(inplace = True, drop = True)

In [ ]:
# Le agregamos la columna ingreso_necesario que nos indica según el hogar, cuánto debe ganar para no ser
# considerado pobre
ingreso_adulto_min = 18914
respondieron["ingreso_necesario"] = ingreso_adulto_min * respondieron["ad_equiv_hogar"]

## 9

In [ ]:
# Creamos una columna dummy, que dice si el individuo es pobre o no. Depende de si sus ingresos
# son mayores a ingreso_necesario
respondieron["pobre"] = (respondieron["ITF"] < respondieron["ingreso_necesario"]).astype(int)

## 10

In [ ]:
# Agarramos solo GBA, porque pide calcularlo para GBA
eph_gba = respondieron[respondieron["AGLOMERADO"] == 33]

In [ ]:
#Agrupamos por hogar con el groupby("CODUSU") y nos quedamos con el PONDIH, y la columna de pobre,
#para las cuales calculamos la media
eph_gba = eph_gba.groupby("CODUSU").agg({"PONDIH":"mean","pobre":"mean"})

In [ ]:
# Armamos la suma de los PONDIH para pobres y para todos
pondih_pobres = eph_gba[eph_gba["pobre"] == 1].apply({"PONDIH":"sum"})
pondih_total = eph_gba.apply({"PONDIH":"sum"})

In [ ]:
# Por último, los dividimos para así obtener la tasa de hogares pobres
pondih_pobres/pondih_total

Por lo tanto, la tasa de hogares bajo la liea de pobreza para el GBA es del 36,5%. Por su parte, para el periodo que estamos analizando el Indec reporta que el porcentaje de hogares por debajo de la línea de pobreza (LP) alcanzó el 31,2%; en los cuales reside el 40,6% de las personas.

# Parte II

## 1

In [ ]:
def evalua_metodo(X_train, X_test, y_train, y_test, modelo, parametro=1.0, n=5):
    '''
    Función que reporta métricas de evaluación de un modelo a elección.
    
    input:
        X_train: X de entrenamiento.
        X_test: X de evaluación.
        y_train: Y de entrenamiento.
        y_test: Y de evaluación.
        modelo: Elección de modelo, default = Logit. (Logit = "Logit", Lineal = "Lineal", KNN = "KNN", Lasso = "Lasso",
                                                      Ridge = "Ridge", Elastic net = "ElasticNet", CART = "CART", 
                                                      SVM = "SVM", Bagging = "Bagging", Random Forest = "RandomForest", 
                                                      Boosting = Boosting").
        parametro: Parámetro de regularización de los modelos. Por default = 1.0
        n: Parámetro para definir la catnidad de n-neighbours en KNN. Por default = 5.
        
    output:
        Df con las columnas: (modelo, ecm, parámetro, accuracy, matriz_confusion, auc)
        coe: En el caso de un modelo "Lasso", se define una variable global con los coeficientes de las variables X.
        
    '''
    global coe
    global coef2
    ev_modelo = pd.DataFrame(columns=["modelo", "ecm", "parámetro", "auc", "accuracy", "fpr", "tpr"])
    
    # Filtro para distintos modelos.
    pob = LogisticRegression(max_iter = 5000, fit_intercept = False)
    if modelo == "Lineal":
        pob = LinearDiscriminantAnalysis()
    elif modelo == "KNN":
        pob = KNeighborsClassifier(n_neighbors = int(n))
    elif modelo == "Logit":
        pob = LogisticRegression(max_iter = 5000, fit_intercept = False)
    elif modelo == "Lasso":
        pob = LogisticRegression(penalty = "l1", max_iter = 5000, 
                                 fit_intercept = False, solver = "liblinear", C = parametro)
    elif modelo == "Ridge":
        pob = LogisticRegression(penalty = "l2", max_iter = 5000, 
                                 fit_intercept = False, solver = "liblinear", C = parametro)
    elif modelo == "ElasticNet":
        pob = LogisticRegression(penalty = "elasticnet", max_iter = 5000, 
                                 fit_intercept = False, solver = "saga", C = parametro)
    elif modelo == "CART":
        pob = DecisionTreeClassifier(ccp_alpha = parametro)
    elif modelo == "SVM":
        pob = SVC(C = parametro,gamma='auto')
    elif modelo == "Bagging":
        pob = BaggingClassifier(base_estimator=SVC(C = parametro))
    elif modelo == "RandomForest":
        pob = RandomForestClassifier(ccp_alpha = parametro)
    elif modelo == "Boosting":
        pob = GradientBoostingClassifier(learning_rate=1.0, ccp_alpha = parametro)
    
    # Ajustamos el modelo        
    pob = pob.fit(X_train, y_train)     
    
    # Luego predecimos y con las X de evaluación.
    y_pred = pob.predict(X_test)
    
    if modelo != "SVM":
        # Probabilidad de que Y tome valor 1 segun el valor de X 
        y_pred_proba = pob.predict_proba(X_test)[:,1]

    #Valores de Accuracy
    accuracy_pob = accuracy_score(y_test, y_pred)

    #Matriz de confusión
    matriz_confusion = confusion_matrix(y_test, y_pred)

    if modelo != "SVM":
        #Valores de AUC
        auc = roc_auc_score(y_test, y_pred_proba)
        fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

    if modelo != "SVM":
        #Curva ROC
        display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=auc, estimator_name='Reg_log')
        def ROC(roc):
            display.plot()  
            plt.plot([0, 1], [0, 1], color='red', linestyle='--')
            plt.show()
        ROC(display)

    # Error cuadrático medio
        
    ecm = mean_squared_error(y_pred, y_test)
    
    if modelo != "SVM":
        ev_modelo = ev_modelo.append({"modelo": modelo, "ecm": ecm, 
                                      "parámetro":parametro, "auc":auc,
                                      "accuracy":accuracy_pob, "fpr":fpr, "tpr":tpr}, ignore_index=True)
    elif modelo == "SVM":
        ev_modelo = ev_modelo.append({"modelo": modelo, "ecm": ecm, 
                                      "parámetro":parametro,
                                      "accuracy":accuracy_pob}, ignore_index=True)
    if modelo == "Lasso":
        coe = pob.coef_
        coef2 = pd.DataFrame(coef, index=X_test.columns)
    
    return ev_modelo

<span style="color:red"> (Rodrigo) Esta función debería hacerse lo más genérica posible. Notar que el código para ajustar un modelo es el mismo en esencia. Podemos recibir como argumento un modelo genérico, utilizar el método fit para ajustarlo, y calcular las métricas de interés de la misma manera sin importar de qué modelo se trate. Es decir que no es necesario utilizar sentencias condicionales para particularizar para cada modelo.</span>

## 2

In [ ]:
def cross_validation(x, y, k, modelo, parametro=1.0):
    '''
    Función que realiza validación cruzada con k iteraciones para cierto modelo.
    
    input:
        x: Variables explicativas.
        y: Variable dependiente.
        k: Cantidad de particiones de los datos.
        modelo: Elección de modelo, default = Logit. (Logit = "Logit", Lineal = "Lineal", KNN = "KNN", Lasso = "Lasso",
                                                      Ridge = "Ridge", Elastic net = "ElasticNet", CART = "CART", 
                                                      SVM = "SVM", Bagging = "Bagging", Random Forest = "RandomForest", 
                                                      Boosting = Boosting").
        parametro: Parámetro de regularización de los modelos. Por default = 1.0
        n: Parámetro para definir la catnidad de n-neighbours en KNN. Por default = 5.
        
    output:
        Df con las columnas: (modelo, particion, ecm, accuracy, matriz_confusion, auc, fpr, tpr)
        coe_cv: En el caso de un modelo "Lasso", se define un df global con los coeficientes de las variables X para las 
                k particiones por CV.
    '''
    global coe_cv
    
    if modelo == "Lasso":
        coe_cv = pd.DataFrame(columns=["parámetro","coeficientes"])
    
    # Creo el df ecms
    ecms = pd.DataFrame(columns=["modelo", "particion", "ecm", "parámetro", "auc", "accuracy", "fpr", "tpr"])
    
    kf = KFold(n_splits = k, shuffle = True)
    
    for i, (train_index, test_index) in enumerate(kf.split(x)):
        x_train, x_test = x.iloc[list(train_index)], x.iloc[list(test_index)]
        y_train, y_test = y.iloc[list(train_index)], y.iloc[list(test_index)]
        
        ev_modelo = evalua_metodo(x_train, x_test, y_train, y_test, modelo, parametro)
        
        ecms = ecms.append({"modelo":ev_modelo["modelo"].item(), 
                            "particion": i, "ecm":ev_modelo["ecm"].item(), 
                            "parámetro":parametro, "auc":ev_modelo["auc"].item(),
                            "accuracy":ev_modelo["accuracy"].item(), "fpr":ev_modelo["fpr"].item(),
                            "tpr":ev_modelo["tpr"].item()}, ignore_index=True)
        
        if modelo == "Lasso":
            coe_cv = coe_cv.append({"particion":i,"parámetro":parametro,"coeficientes":coe}, ignore_index=True)

        ecms = ecms.astype({"particion":int})
        
    return ecms 

## 3

In [ ]:
def evalua_config(lista_parametros, x, y, k, modelo):
    '''
    Función que selecciona los hiperparámetros que menor ECM generen, mediante Cross-Validation.
    
    input:
        lista_parametros: lista de parámetros de regularización de los modelos que se van a usar para evaluar.
        x: Variables explicativas.
        y: Variable dependiente.
        k: Cantidad de particiones de los datos.
        modelo: Elección de modelo, default = Logit. (Logit = "Logit", Lineal = "Lineal", KNN = "KNN", Lasso = "Lasso",
                                                      Ridge = "Ridge", Elastic net = "ElasticNet", CART = "CART", 
                                                      SVM = "SVM", Bagging = "Bagging", Random Forest = "RandomForest", 
                                                      Boosting = Boosting").

    output:
        todos: df que contiene la información del modelo utilizado con las columnas
               (modelo, particion, ecm, parámetro, accuracy, auc, fpr, tpr).
        ecms_avg: df que hace el promedio por parámetro, con las columnas 
                  (modelo, particion, ecm, parámetro, accuracy, auc, fpr, tpr)
        ev_modelo: df que contiene la información del mejor modelo elegido por ECM, con las columnas 
                  (modelo, particion, ecm, parámetro, accuracy, auc, fpr, tpr)
        coe_cvs: En el caso de un modelo "Lasso", se define un df global con los coeficientes de las variables X para las 
                k particiones por CV.
    '''
    global coe_cvs
    
    # Defino data frames para usar después
    
    todos = pd.DataFrame(columns=["modelo", "particion", "ecm", "parámetro", "auc", "accuracy", "fpr", "tpr"])
    if modelo == "Lasso":
        coe_cvs = pd.DataFrame(columns=["parámetro","coeficientes"])
    
    # For loop que para cada parámetro corre Cross validation con k particiones.
    for i in lista_parametros:
        ecms = cross_validation(x, y, k, modelo, i)
        # ecmss es un df que guarda los resultados de todos los parámetros k veces.
        todos = todos.append(ecms)
        if modelo == "Lasso":
            coe_cvs = coe_cvs.append(coe_cv)
    
    # ecms_avg es un df que guarda los resultados de todos los parámetros haciendo un promedio de todos los ECM de ecmss
    ecms_avg = todos.groupby("parámetro").agg({"ecm":"mean", "modelo":"max", "auc":"mean",
                                               "accuracy":"mean"}).reset_index()
    
    min_ecm = np.Inf
    mejor_parametro = None
    # For loop que permite determinar el parámetro que tiene el menor ECM en promedio.
    for index, row in ecms_avg.iterrows():
        if row["ecm"] < min_ecm:
            min_ecm = row["ecm"]
            mejor_parametro = float(row["parámetro"])
    print("El minimo error es ", min_ecm, " y se da con un parametro de ", mejor_parametro)
    
    # Corro el modelo con el parámetro seleccionado con menor ECM en promedio.
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
    ev_modelo = evalua_metodo(X_train, X_test, y_train, y_test, modelo, mejor_parametro)
    
    return (todos, ecms_avg, ev_modelo)

<span style="color:red"> (Rodrigo) Para esta función, la idea era que reciba una lista de configuraciones de hiperparámetros, donde las configuraciones debían adaptarse a distintos modelos (y los hiperparámetros podían ser más de uno). <br><br>
Una forma de hacerlo es hacer que la función reciba además el modelo sin inicializar, es decir el nombre del modelo sin paréntesis, como se hizo en la tutorial 7, por ejemplo `LogisticRegression` <br>
Entonces, se iteraría con un `for` sobre los distintos diccionarios de configuraciones y antes de cada llamada a `cross_validation` o `evalua_metodo` se inicializaría el modelo con los hiperparámetros particulares de esa iteración. Sería algo así:
```    
# este es un ejemplo, pero en realidad configs se debe definir 
# por fuera de evalua_config (en particular, en evalua_multiples_metodos) 
# y evalua_config debe recibir algo similar a configs como argumento.

configs = [{ "penalty": "l1", "C"=0.1 }, { "penalty": "l2", "C"=0.001 }]

for config in configs:
    modelo = LogisticRegression
    modelo = modelo(**configuracion) # inicializamos el modelo con los hiperparámetros
    ..., ..., ... = cross_validation(modelo, ...) # le paso a cross_validation el modelo inicializado
```
</span>

## 4

In [ ]:
def evalua_multiples_metodos(lista_parametros, x, y, k):
    '''
    Función que corre múltiples métodos para diferentes parámetros los hiperparámetros que menor ECM generen, 
    mediante Cross-Validation.
    
    input:
        lista_parametros: lista de parámetros de regularización de los modelos que se van a usar para evaluar.
        x: Variables explicativas.
        y: Variable dependiente.
        k: Cantidad de particiones de los datos.

    output:
        lista_todos: df que contiene la información de todos los modelos utilizados con las columnas
               (modelo, particion, ecm, parámetro, accuracy, auc, fpr, tpr).
        lista_promedios: df que hace el promedio por parámetro de los diferentes modelos, con las columnas 
                  (modelo, particion, ecm, parámetro, accuracy, auc, fpr, tpr)
        lista_mejores: df que contiene la información de los mejores modelos elegido por ECM en los distintos modelos,
                       con las columnas 
                  (modelo, particion, ecm, parámetro, accuracy, auc, fpr, tpr)
    '''
    
    lista_todos = pd.DataFrame(columns=["modelo", "particion", "ecm", "parámetro", "auc", "accuracy", "fpr", "tpr"])
    lista_promedios = pd.DataFrame(columns=["modelo", "parámetro", "ecm", "auc", "accuracy"])
    lista_mejores = pd.DataFrame(columns=["modelo", "ecm", "parámetro", "auc", "accuracy", "fpr", "tpr"])
    
    modelos = ["Lineal", "KNN", "Logit", "CART", "SVM", "Bagging", "RandomForest", "Boosting", "Lasso", "Ridge"]
    for modelo in modelos:
        todos, ecms_avg, ev_modelo = evalua_config(lista_parametros, x, y, k, modelo)
        lista_todos = lista_todos.append(todos)
        lista_promedios = lista_promedios.append(ecms_avg)
        lista_mejores = lista_mejores.append(ev_modelo)
        
    return (lista_todos, lista_promedios, lista_mejores)

<span style="color:red"> (Rodrigo) En esta deberían definirse los distintos modelos y respectivos hiperparámetros que se van a probar en las funciones internas. Se puede usar un diccionario anidado como este:</span>

```
modelos = {
    "regresion_logistica": { 
        "modelo": LogisticRegression, 
        "hiperparametros": [{ "penalty": "l1", "C"=0.1 }, { "penalty": "l2", "C"=0.001 }],
    "vecinos_cercanos": { 
        "modelo": KNearestNeighbors, 
        "hiperparametros": [{"n_neighbors": n} for range(3,10)] 
    },
}
```
<span style="color:red">Lo que uso para definir los hiperparámetros de KNN es una técnica llamada "definición de listas por comprensión", lo pueden mirar en la tutorial 7. Luego pueden iterar sobre este diccionario de esta manera</span>

```
for nombre_modelo in modelos:
    modelo = modelos[nombre_modelo]["modelo"] # esto puede valer, por ejemplo, LogisticRegression
    configs = modelos[nombre_modelo]["hiperparametros"] # esta es la lista de configs. que voy a probar
    
    # agrego nombre_modelo a evalua_config como argumento 
    # porque puede resultar útil si se lo quiere agregar como columna de un dataframe
    ... = evalua_config(nombre_modelo, modelo, configs, ...) 

```

# Parte III

# 1 

In [ ]:
#eliminamos todas las variables relacionadas a ingresos. 
#respondieron.drop(['ITF','DECIFR', 'RDECIFR', 'GDECCFR', 'ADECIFR', 'IPCF', 'DECCFR', 'RDECCFR','GDECCFR', 'ADECCFR', 'PONDIH'], axis=1, inplace=True)
#norespondieron.drop(['ITF','DECIFR', 'RDECIFR', 'GDECCFR', 'ADECIFR', 'IPCF', 'DECCFR', 'RDECCFR','GDECCFR', 'ADECCFR', 'PONDIH'], axis=1, inplace=True)
#'IDECCFR', 'PDECIFR', 'PDECCFR'
#estas no las encontro

In [ ]:
# Eliminamos de ambas bases las columnas de ingresos
respondieron.drop(["PP08D1", "PP08D4", "PP08F1", "PP08F2", "PP08J1", "PP08J2", "PP08J3", "P21", "TOT_P12", "P47T",\
                  "V2_M", "V3_M", "V4_M", "V5_M", "V8_M", "V9_M", "V10_M", "V11_M", "V12_M", "V18_M", "V19_AM",\
                   "V21_M", "T_VI", "ITF", "IPCF", "ingreso_necesario", "ad_equiv_hogar", "adulto_equiv", "IX_MAYEQ10"], axis = 1, inplace = True)
norespondieron.drop(["PP08D1", "PP08D4", "PP08F1", "PP08F2", "PP08J1", "PP08J2", "PP08J3", "P21", "TOT_P12", "P47T",\
                  "V2_M", "V3_M", "V4_M", "V5_M", "V8_M", "V9_M", "V10_M", "V11_M", "V12_M", "V18_M", "V19_AM",\
                    "V21_M", "T_VI", "ITF", "IPCF", "ad_equiv_hogar", "adulto_equiv", "IX_MAYEQ10"], axis = 1, inplace = True)
# Nos dimos cuenta tarde que faltan cambiar algunas para que no aparezcan NaN y dropear las de deciles
respondieron["CH14"].fillna(99, inplace = True) #Muchos no respondieron entonces reemplazamos los missing por 99. 
norespondieron["CH14"].fillna(99, inplace = True)
respondieron.drop(["CH15_COD", "CH16_COD", "PP03C", "PP03D", "CODUSU"], axis = 1, inplace = True) # innecesarios (con muy pocas o ninguna observación) o de ingreso 
respondieron.drop(["DECOCUR", "RDECOCUR", "GDECOCUR", "ADECOCUR", "PONDIIO", "DECIFR", "RDECIFR", "GDECIFR",\
                   "PONDII", "PONDIH", "PONDERA", "ADECIFR", "DECCFR", "RDECCFR", "GDECCFR",\
                   "ADECCFR", "DECINDR", "RDECINDR", "GDECINDR", "ADECINDR", "CH05"], axis = 1, inplace = True) # todas deciles
norespondieron.drop(["CH15_COD", "CH16_COD", "PP03C", "PP03D", "CODUSU"], axis = 1, inplace = True) # innecesarios (con muy pocas o ninguna observación) o de ingreso 
norespondieron.drop(["DECOCUR", "RDECOCUR", "GDECOCUR", "ADECOCUR", "PONDIIO", "DECIFR", "RDECIFR", "GDECIFR",\
                   "PONDII", "PONDIH", "PONDERA", "ADECIFR", "DECCFR", "RDECCFR", "GDECCFR",\
                   "ADECCFR", "DECINDR", "RDECINDR", "GDECINDR", "ADECINDR", "CH05"], axis = 1, inplace = True) # todas deciles
respondieron.reset_index(inplace = True, drop = True)
norespondieron.reset_index(inplace = True, drop = True)

In [ ]:
# Creamos una copia del dataframe para luego recuperar las columnas que no son categóricas
respondieron_temp = respondieron.copy()
norespondieron_temp = norespondieron.copy()

In [ ]:
# Herramienta para poder utilizar variables categóricas en los modelos.
label_encoder = preprocessing.LabelEncoder()

In [ ]:
# Le aplicamos la herramienta al df
respondieron = respondieron.apply(label_encoder.fit_transform)
norespondieron = norespondieron.apply(label_encoder.fit_transform)

In [ ]:
# Volvemos a introducir las columnas no categóricas 
respondieron[["PP3E_TOT", "PP3F_TOT", "PP04B2", "PP04B3_MES", "PP04B3_ANO", "PP04B3_DIA", \
              "PP05B2_MES", "PP05B2_ANO", "PP05B2_DIA", "PP11B2_MES", "PP11B2_ANO", "PP11B2_DIA",
              "IX_TOT", "IX_MEN10"]] \
= respondieron_temp[["PP3E_TOT", "PP3F_TOT", "PP04B2", "PP04B3_MES", "PP04B3_ANO", "PP04B3_DIA", \
                     "PP05B2_MES", "PP05B2_ANO", "PP05B2_DIA", "PP11B2_MES", "PP11B2_ANO", "PP11B2_DIA",
                     "IX_TOT", "IX_MEN10"]] 
norespondieron[["PP3E_TOT", "PP3F_TOT", "PP04B2", "PP04B3_MES", "PP04B3_ANO", "PP04B3_DIA", \
              "PP05B2_MES", "PP05B2_ANO", "PP05B2_DIA", "PP11B2_MES", "PP11B2_ANO", "PP11B2_DIA","IX_TOT", "IX_MEN10"]] \
= norespondieron_temp[["PP3E_TOT", "PP3F_TOT", "PP04B2", "PP04B3_MES", "PP04B3_ANO", "PP04B3_DIA", \
                     "PP05B2_MES", "PP05B2_ANO", "PP05B2_DIA", "PP11B2_MES", "PP11B2_ANO", "PP11B2_DIA",
                      "IX_TOT", "IX_MEN10"]]

In [ ]:
#Agregamos la columna de 1 
respondieronx = respondieron.iloc[:, :-1]
respondieronx["inter"] = 1
norespondieron["inter"] = 1

## 2

In [ ]:
# Corremos evalua_multiples_metodos con el hiperparámetro 2
lista = [2]
todo, promedio, mejor = evalua_multiples_metodos(lista,respondieronx,respondieron["pobre"],2)

In [ ]:
promedio

## 3

λ lo elegiríamos por cross validation ya que queremos minimizar el error de pronostico por fuera de la muestra. Por lo tanto, debe estimarse el modelo para muchos valores de λ alternativos y calcular el error de pronóstico por cross validation. Para esto
primero hay qye partir los datos al azar en K partes, luego ajustar el modelo dejando afuera una de las particiones y, por último, Computar el error de predicción para los datos no utilizados. Esto debe repetirse para k = 1,...,K. 
Una vez hecho esto, vamos a elegir el valor de λ que genere la mejor capacidad predictiva, es decir, el que minimiza el error de pronóstico fuera de la muestra computado por cross validation. Este es el sentido en el cual el modelo esta aprendiendo, dado  que estamos haciendo que los datos elijan el λ óptimo.


Para la eleccion no debe usarse el conjunto de prueba dado que no podríamos comparar el desempeño con otros modelos dado que todos los modelos se estiman con los datos de entrenamiento y luego se evalúan con los datos de test. Por lo tanto, si en un modelo a λ lo computamos usando los datos de test, deja de ser comparable con el resto de los modelos, ya que tendría una ventaja por sobre todos los demás. 


<span style="color:red">
(Rodrigo) Lo que escribieron es correcto, pero dejo la siguiente explicación que escribí para otro grupo para asegurarme que el concepto queda claro:<br><br>
Es importante que, al elegir un hiperparámetro de un modelo, todo el proceso de CV ocurra dentro del conjunto de entrenamiento.
Supongamos que tenemos 100 observaciones, por poner un número redondo, y separamos 80 para training y 20 para testing. Queremos usar las 80 observaciones para elegir y ajustar el mejor modelo posible (esto incluye elegir los mejor hiperparámetros), y las 20 restantes para evaluar qué tan bien predice ese modelo que generamos.</span>
<span style="color:red"> <br>
Ahora imaginemos que tenemos una familia de modelos, como ocurre en regresión logística (donde cada miembro de la familia corresponde a un conjunto distinto de hiperparámetros). Entonces para lo que usamos k-fold CV es para elegir al mejor miembro de esta familia, es decir al mejor conjunto de hiperparámetros. Para eso, con cada configuración de hiperparámetros (supongamos que k = 5), separamos 16 observaciones y ajustamos el modelo en las 64 restantes, y así vamos cambiando las 16 observaciones que dejamos afuera, y en cada caso obtenemos un modelo ajustado distinto. (Estas 16 observaciones se las suele denominar de validación para distinguirlas de las 20 de testing, que no deben participar de este proceso para nada). Luego promediamos los ECM obtenidos para cada partición para obtener un mejor estimado. En base a este estimado elegimos el hiperparámetro.
Ahora, una vez que elegimos el hiperparámetro, es mejor ajustar el modelo de nuevo pero ahora sobre las 80 observaciones, y usar los 20 restantes (las de testing) para reportar las métricas.</span>

## 4 

El problema de usar un K muy pequeño es que a pesar de que maximiza los datos para estimar, es sensible a los valores particulares para la evaluación. Por lo que el error de pronostico depende mucho de lo que pase con los pocos datos de test.
Por otro lado, el problema de usar un muy K grande es que a pesar de que maximiza los datos para evaluar, el modelo es estimado menos precisamente dado que se usan menos datos de entrenamiento.
Por último, cuando K = N se va dejando de lado una observación por vez. Así, se estima el modelo n veces con n − 1 datos (‘leave one out’)




<span style="color:red">
(Rodrigo) La afirmación de que para K grande se usan menos datos para entrenar no es correcta, de hecho lo contrario lo es. Ejemplo: si K=5, se entrena con el 80% de las observaciones cada vez; si K=10, se usa el 90%. El problema de usar un K grande viene dado por el número de veces que se tiene que entrenar el modelo, lo cual puede resultar muy costoso computacionalmente sin un beneficio asociado que lo justifique. Como bien dicen, hacer K=N implica entrenar el modelo N veces con N-1 datos. Si la base es muy grande, esto puede ser muy costoso de hacer computacionalmente, sumado a que no hay mucha ganancia ya que ya se obtendría una performance similar entrenando con el 80 ó 90% de los datos. Puede tener sentido hacer K=N sentido si la base es muy chica y queremos quitar tan pocas observaciones como podamos a la hora de entrenar. De lo contrario, un K del orden de 5 ó 10 es en general deseable.
</span>

## 5

In [ ]:
# Definimos la lista de parámetros a evaluar en Lasso y Ridge.
lista = 10**np.linspace(-5,5,11)
todos_l, promedio_l, mejor_l = evalua_config(lista, respondieronx,respondieron["pobre"],10,"Lasso")
todos_r, promedio_r, mejor_r = evalua_config(lista, respondieronx,respondieron["pobre"],10,"Ridge")

El λ elegido en regresión logísitica con Ridge y Lasso es 100. 

In [ ]:
# Creamos las variables en las cuales almaceno su ecm por parámetro, para las diferentes particiones
for i, x in zip(lista, range(1,12,1)):
     locals()["l_"+str(x)] = todos_l[todos_l["parámetro"]==i].ecm
for i, x in zip(lista, range(1,12,1)):
     locals()["r_"+str(x)] = todos_r[todos_r["parámetro"]==i].ecm

In [ ]:
# Definimos una variable con todas las variables con los ecm por parámetro para hacer el boxplot
data_l = (l_1,l_2,l_3,l_4,l_5,l_6,l_7,l_8,l_9,l_10,l_11)
data_r = (r_1,r_2,r_3,r_4,r_5,r_6,r_7,r_8,r_9,r_10,r_11)

In [ ]:
# Boxplot para Lasso
fig, ax = plt.subplots(figsize =(15, 7))
ax.xaxis.set(ticklabels=[0.00001,0.0001,0.001,0.01,0.1,1,10,100,1000,10000,100000])
ax.set(title="ECM por parámetro para Lasso" , ylabel= "ECM", xlabel= "Parámetro")
ax.boxplot(data_l)
plt.show()

In [ ]:
# Boxplot para Ridge
fig, ax = plt.subplots(figsize =(15, 7))
ax.xaxis.set(ticklabels=[0.00001,0.0001,0.001,0.01,0.1,1,10,100,1000,10000,100000])
ax.set(title="ECM por parámetro para Ridge" , ylabel= "ECM", xlabel= "Parámetro")
ax.boxplot(data_r)
plt.show()

In [ ]:
#Dividimos el df coe_cvs por parámetro
coeficientes_1 = coe_cvs[coe_cvs["parámetro"]== 0.00001]["coeficientes"]
coeficientes_1.reset_index(inplace = True, drop = True)
coeficientes_2 = coe_cvs[coe_cvs["parámetro"]== 0.0001]["coeficientes"]
coeficientes_2.reset_index(inplace = True, drop = True)
coeficientes_3 = coe_cvs[coe_cvs["parámetro"]== 0.001]["coeficientes"]
coeficientes_3.reset_index(inplace = True, drop = True)
coeficientes_4 = coe_cvs[coe_cvs["parámetro"]== 0.01]["coeficientes"]
coeficientes_4.reset_index(inplace = True, drop = True)
coeficientes_5 = coe_cvs[coe_cvs["parámetro"]== 0.1]["coeficientes"]
coeficientes_5.reset_index(inplace = True, drop = True)
coeficientes_6 = coe_cvs[coe_cvs["parámetro"]== 1.0]["coeficientes"]
coeficientes_6.reset_index(inplace = True, drop = True)
coeficientes_7 = coe_cvs[coe_cvs["parámetro"]== 10.0]["coeficientes"]
coeficientes_7.reset_index(inplace = True, drop = True)
coeficientes_8 = coe_cvs[coe_cvs["parámetro"]== 100.0]["coeficientes"]
coeficientes_8.reset_index(inplace = True, drop = True)
coeficientes_9 = coe_cvs[coe_cvs["parámetro"]== 1000.0]["coeficientes"]
coeficientes_9.reset_index(inplace = True, drop = True)
coeficientes_10 = coe_cvs[coe_cvs["parámetro"]== 10000.0]["coeficientes"]
coeficientes_10.reset_index(inplace = True, drop = True)
coeficientes_11 = coe_cvs[coe_cvs["parámetro"]== 100000.0]["coeficientes"]
coeficientes_11.reset_index(inplace = True, drop = True)

In [ ]:
# Definimos variables que contengan la cantidad de ceros que tienen las diferentes particiones para poder hacer el boxplot.
for x in range(0,10,1):
    locals()["ceros_"+str(x)] = np.count_nonzero(coeficientes_1[x]==0)
data_1 = (ceros_0, ceros_1, ceros_2, ceros_3, ceros_4, ceros_5, ceros_6, ceros_7, ceros_8, ceros_9)
for x in range(0,10,1):
    locals()["ceros_"+str(x)] = np.count_nonzero(coeficientes_2[x]==0)
data_2 = (ceros_0, ceros_1, ceros_2, ceros_3, ceros_4, ceros_5, ceros_6, ceros_7, ceros_8, ceros_9)
for x in range(0,10,1):
    locals()["ceros_"+str(x)] = np.count_nonzero(coeficientes_3[x]==0)
data_3 = (ceros_0, ceros_1, ceros_2, ceros_3, ceros_4, ceros_5, ceros_6, ceros_7, ceros_8, ceros_9)
for x in range(0,10,1):
    locals()["ceros_"+str(x)] = np.count_nonzero(coeficientes_4[x]==0)
data_4 = (ceros_0, ceros_1, ceros_2, ceros_3, ceros_4, ceros_5, ceros_6, ceros_7, ceros_8, ceros_9)
for x in range(0,10,1):
    locals()["ceros_"+str(x)] = np.count_nonzero(coeficientes_5[x]==0)
data_5 = (ceros_0, ceros_1, ceros_2, ceros_3, ceros_4, ceros_5, ceros_6, ceros_7, ceros_8, ceros_9)
for x in range(0,10,1):
    locals()["ceros_"+str(x)] = np.count_nonzero(coeficientes_6[x]==0)
data_6 = (ceros_0, ceros_1, ceros_2, ceros_3, ceros_4, ceros_5, ceros_6, ceros_7, ceros_8, ceros_9)
for x in range(0,10,1):
    locals()["ceros_"+str(x)] = np.count_nonzero(coeficientes_7[x]==0)
data_7 = (ceros_0, ceros_1, ceros_2, ceros_3, ceros_4, ceros_5, ceros_6, ceros_7, ceros_8, ceros_9)
for x in range(0,10,1):
    locals()["ceros_"+str(x)] = np.count_nonzero(coeficientes_8[x]==0)
data_8 = (ceros_0, ceros_1, ceros_2, ceros_3, ceros_4, ceros_5, ceros_6, ceros_7, ceros_8, ceros_9)
for x in range(0,10,1):
    locals()["ceros_"+str(x)] = np.count_nonzero(coeficientes_9[x]==0)
data_9 = (ceros_0, ceros_1, ceros_2, ceros_3, ceros_4, ceros_5, ceros_6, ceros_7, ceros_8, ceros_9)
for x in range(0,10,1):
    locals()["ceros_"+str(x)] = np.count_nonzero(coeficientes_10[x]==0)
data_10 = (ceros_0, ceros_1, ceros_2, ceros_3, ceros_4, ceros_5, ceros_6, ceros_7, ceros_8, ceros_9)
for x in range(0,10,1):
    locals()["ceros_"+str(x)] = np.count_nonzero(coeficientes_11[x]==0)
data_11 = (ceros_0, ceros_1, ceros_2, ceros_3, ceros_4, ceros_5, ceros_6, ceros_7, ceros_8, ceros_9)

In [ ]:
# Las agrupo en una variable para poder hacer el boxplot
data = (data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,data_10,data_11)

In [ ]:
# Boxplot de variables eliminadas por parámetro para Lasso
fig, ax = plt.subplots(figsize =(15, 7))
ax.xaxis.set(ticklabels=[0.00001,0.0001,0.001,0.01,0.1,1,10,100,1000,10000,100000])
ax.set(title="Variables eliminadas por parámetro en Lasso" , ylabel= "Cant var eliminadas", xlabel= "Parámetro")
ax.boxplot(data)
plt.show()

<span style="color:blue"> (Rodrigo) Excelente, este es el comportamiento esperado. En el límite cuando lambda tiende a cero, es equivalente a entrenar mínimos cuadrados ordinarios (ya que el término de Lasso no tiene peso). En el otro extremo, sólo el término de penalización de Lasso tiene preponderancia en la función de costo del modelo, por lo tanto lo único que "quiere" el modelo es hacer los coeficientes iguales a cero, sin importar la bondad de la predicción.</span>

## 6

In [ ]:
# El mejor modelo.
mejor_l

In [ ]:
# Partimos para poder aplicar la función.
X_train, X_test, y_train, y_test = train_test_split(respondieronx,respondieron["pobre"],test_size=0.3)

In [ ]:
# Corremos el mejor modelo
evalua_metodo(X_train, X_test, y_train, y_test, "Lasso", mejor_l["parámetro"].item())

In [ ]:
# Estas son las variables descartadas
coef2[coef2[0] == 0]

Las variables descartadas son las que hubieramos esperado, ya que creemos que al ver el diseño de registro de la base hogar parecían ser irrelevantes para explicar la tasa de pobreza. Por ejemplo el año de relevamiento (ANO4), la ventana de observación (TRIMESTRE) y el codigo de region (REGION) no parecen variables que tengan una relacion significativa con la tasa de pobreza.
Con respecto a lo respondido en el inciso 1 de la Parte 1, ninguna de las variables que creíamos a priori relevantes para predecir pobreza fueron eliminidas, por lo que efectivamente presentan un poder explicativo significativo. 

## 7

In [ ]:
# Comparamos estos dos modelos con mismo parámetro
promedio_l[promedio_l["parámetro"] == 100].ecm.item()

In [ ]:
# Comparamos estos dos modelos con mismo parámetro
promedio_r[promedio_r["parámetro"] == 100].ecm.item()

Por lo tanto, Lasso funcionó mejor ya que se obtiene un error cuadratico medio menor que en el caso de Ridge. 

## 8

In [ ]:
# Evalúamos todos los modelos dentro de multiples modelos para poder definir cual es el que mejor predice.
todo2, promedio2, mejor2 = evalua_multiples_metodos(lista, respondieronx, respondieron["pobre"], 10)

In [ ]:
mejor2

El método que predice mejor es Boosting con el hiperparámetro 0.00001, dado que presenta el mínimo error cuadrático medio. 

## 9

In [ ]:
ult = GradientBoostingClassifier(learning_rate=1.0, ccp_alpha = 0.00001)
# Ajustamos el modelo        
ult = ult.fit(X_train, y_train)     
    
# Luego predecimos y con las X de evaluación.
y_pred = ult.predict(X_test)

# Probabilidad de que Y tome valor 1 segun el valor de X 
y_pred_proba = ult.predict_proba(X_test)[:,1]

#Valores de Accuracy
accuracy_pob = accuracy_score(y_test, y_pred)

#Matriz de confusión
matriz_confusion = confusion_matrix(y_test, y_pred)

#Valores de AUC
auc = roc_auc_score(y_test, y_pred_proba)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

#Curva ROC
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=auc, estimator_name='Reg_log')
def ROC(roc):
        display.plot()  
        plt.plot([0, 1], [0, 1], color='red', linestyle='--')
        plt.show()
ROC(display)

# Error cuadrático medio
        
ecm = mean_squared_error(y_pred, y_test)

In [ ]:
y_pred_nr = ult.predict(norespondieron)
y_pred_nr.sum()
y_pred_nr.sum()/816

El 43,9% de los hogares son pobres en ests submuestra